# Explore syntax parsing for detection of opinion holders and targets

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
import json

In [4]:
review_data_file = '../lexicon/data/yelp_sample.json'
with open(review_data_file, 'r') as infile:
    R = json.load(infile)

In [16]:
def spacy_sentences(nlp, text):
    sentences = []
    for sent in nlp(text).sents:
        sentences.append(sent)
    return sentences

In [17]:
sample = spacy_sentences(nlp, R[0]['content'])

In [19]:
type(sample[0])

spacy.tokens.span.Span

## Build a graph of the sentences

In [20]:
import networkx as nx
from spacy.displacy import render

In [52]:
test = sample[1]

In [53]:
render(test)

In [54]:
data = pd.DataFrame([{
    'sentence': i, 'token': x.idx, 'text': x.text, 'pos': x.pos_, 'lemma': x.lemma_,
    'dep': x.dep_, 'head': x.head.idx
} for i, x in enumerate(test)])

In [55]:
data

,sentence,token,text,pos,lemma,dep,head
0,0,74,This,DET,this,det,79
1,1,79,location,NOUN,location,nsubj,88
2,2,88,was,AUX,be,ROOT,88
3,3,92,clean,ADJ,clean,acomp,88
4,4,98,with,ADP,with,prep,92
5,5,103,great,ADJ,great,amod,109
6,6,109,service,NOUN,service,pobj,98
7,7,117,and,CCONJ,and,cc,109
8,8,121,food,NOUN,food,conj,109
9,9,126,served,VERB,serve,advcl,88


## Explore the graph for detecting the target of adjectives and verbs
- hints: create a statistics of the adjectives and verbs most common dependencies

In [56]:
token_map = lambda span: dict([(x.idx, x) for x in span])

In [57]:
def explore(token, children=None, level=0, order=None):
    if children is None:
        children = []
    if order is None:
        order = token.idx
    for child in token.children:
        children.append((child, level, child.idx < order))
        explore(child, children=children, level=level+1, order=order)
    return children

In [58]:
m = token_map(test)

In [60]:
explore(m[109])

[(great, 0, True), (and, 0, False), (food, 0, False)]

## Explore the graph for detecting the subjects of verbs

## Create communities of nodes

## Find meaningful pairs ADJ-NOUN per business category